In [ ]:
#!/usr/bin/env python
# coding: utf-8
import numpy as np
from itertools import product
np.set_printoptions(precision=5, suppress=True)
import time

import pandas as pd
import dgl
from dgl.data import DGLDataset
import torch
import os
import numpy as np
import torch
import torch as tf

### 数据生成的一些函数

In [ ]:
def Feasible_Loc_Init(Cur_loc, Size_area, Dist_TX_RX):
    temp_dist = Dist_TX_RX * (np.random.rand(1, 2) - 0.5)
    temp_chan = Cur_loc + temp_dist
    while (np.max(abs(temp_chan)) > Size_area / 2) | (np.linalg.norm(temp_dist) > Dist_TX_RX):
        temp_dist = Dist_TX_RX * (np.random.rand(1, 2) - 0.5)
        temp_chan = Cur_loc + temp_dist
    return temp_chan


def Feasible_Loc_Update(Cur_RX_loc, Cur_TX_loc, Size_area, Dist_TX_RX, Delta_mov):
    temp_chan = 0
    temp_dist = 2 * Dist_TX_RX

    while (np.max(abs(temp_chan)) > Size_area / 2) | (np.linalg.norm(temp_dist) > Dist_TX_RX):
        temp_dir = np.random.rand()
        temp_dist_delta = [Delta_mov * np.cos(2 * np.pi * temp_dir), Delta_mov * np.sin(2 * np.pi * temp_dir)]
        temp_chan = Cur_RX_loc + temp_dist_delta
        temp_dist = Cur_TX_loc - temp_chan

    return temp_chan


def loc_init(Size_area, Dist_TX_RX, Num_D2D, Num_Ch):
    tx_loc = Size_area * (np.random.rand(Num_D2D, 2) - 0.5)
    rx_loc = np.zeros((Num_D2D + 1, 2))
    for i in range(Num_D2D):
        temp_chan = Feasible_Loc_Init(tx_loc[i, :], Size_area, Dist_TX_RX)
        rx_loc[i, :] = temp_chan
    tx_loc_CUE = Size_area * (np.random.rand(Num_Ch, 2) - 0.5)

    return rx_loc, tx_loc, tx_loc_CUE


def loc_update(Size_area, Dist_TX_RX, rx_loc, tx_loc, tx_loc_CUE, Delta_mov):
    tx_loc_update = tx_loc
    rx_loc_update = rx_loc
    tx_loc_CUE_update = tx_loc_CUE

    Num_D2D = np.shape(tx_loc)[0]
    Num_CH = np.shape(tx_loc_CUE)[0]

    ## Determine the location of D2D users
    for i in range(Num_D2D):
        ## Use 2*size_area to deactivate the second condition
        tx_loc_update[i, :] = Feasible_Loc_Update(tx_loc[i, :], tx_loc_update[i, :], Size_area, 2 * Size_area,
                                                  Delta_mov)
        rx_loc_update[i, :] = Feasible_Loc_Update(rx_loc[i, :], tx_loc_update[i, :], Size_area, Dist_TX_RX, Delta_mov)

    ## Determine the location of CUE Users
    for i in range(Num_CH):
        tx_loc_CUE_update[i, :] = Feasible_Loc_Update(tx_loc_CUE[i, :], tx_loc_CUE[i, :], Size_area, 2 * Size_area,
                                                      Delta_mov)

    return rx_loc_update, tx_loc_update, tx_loc_CUE_update


def ch_gen(Size_area, D2D_dist, Num_D2D, Num_Ch, Num_samples, Delta_mov=0.0833, PL_alpha=38., PL_const=34.5):
    ch_w_fading = []

    ## Perform initialization just once and the rest channel is generated by moving users
    rx_loc, tx_loc, tx_loc_CUE = loc_init(Size_area, D2D_dist, Num_D2D, Num_Ch)

    for i in range(Num_samples):
        
        #rx_loc, tx_loc, tx_loc_CUE = loc_update(Size_area, D2D_dist, rx_loc, tx_loc, tx_loc_CUE, Delta_mov)
        
        if i % 1 == 0:
            rx_loc, tx_loc, tx_loc_CUE = loc_init(Size_area, D2D_dist, Num_D2D, Num_Ch)

        ch_w_temp_band = []
        for j in range(Num_Ch):
            tx_loc_with_CUE = np.vstack((tx_loc, tx_loc_CUE[j]))
            ## generate distance_vector
            dist_vec = rx_loc.reshape(Num_D2D + 1, 1, 2) - tx_loc_with_CUE
            dist_vec = np.linalg.norm(dist_vec, axis=2)
            dist_vec = np.maximum(dist_vec, 5)

            # find path loss // shadowing is not considered
            pu_ch_gain_db = - PL_const - PL_alpha * np.log10(dist_vec)
            pu_ch_gain = 10 ** (pu_ch_gain_db / 10)

            multi_fading = 0.5 * np.random.randn(Num_D2D + 1, Num_D2D + 1) ** 2 + 0.5 * np.random.randn(Num_D2D + 1,
                                                                                                        Num_D2D + 1) ** 2
            final_ch = np.maximum(pu_ch_gain * multi_fading, np.exp(-30))
            ch_w_temp_band.append(np.transpose(final_ch))

        ch_w_fading.append(ch_w_temp_band)
    return ch_w_fading


def cal_SINR_one_sample_one_channel(channel, tx_power, noise):
    ## Note that we transpose the channel to
    diag_ch = np.diag(channel)
    inter_ch = channel-np.diag(diag_ch)
    tot_ch = np.multiply(channel, np.expand_dims(tx_power, -1))
    int_ch = np.multiply(inter_ch, np.expand_dims(tx_power, -1))
    sig_ch = np.sum(tot_ch-int_ch, axis=1)
    int_ch = np.sum(int_ch, axis=1)

    SINR_val = np.divide(sig_ch, int_ch+noise)
    cap_val = np.log(1.0+SINR_val)
    return cap_val




def temp_power_function(granuity, num_channel):
    returned_array = []
    total_iteration = int(granuity * num_channel + 1)
    returned_array.append(np.zeros((num_channel,)))
    for i in range(total_iteration - 1):
        temp_iter = np.zeros((num_channel,))
        selected_chanel = i // granuity
        selected_power_val = i % granuity
        temp_iter[selected_chanel] = (selected_power_val + 1) / granuity
        returned_array.append(temp_iter)
    return np.array(returned_array)


def all_possible_tx_power(num_channel, num_user, granuty):
    possible_power_for_one_user = np.arange(num_channel * granuty + 1)
    a = np.arange(num_channel * granuty + 1) + 1
    all_possible_int = np.array(list(product(a, repeat=num_user))) - 1
    return temp_power_function(granuty, num_channel)[all_possible_int]


def optimal_power(channel, tx_max, granuty, noise, DUE_thr, CUE_thr, tx_power_set):
    num_channel = channel.shape[1]
    ## Note that the num_user i
    num_D2D_user = channel.shape[2] - 1
    num_samples = channel.shape[0]
    tot_cap = 0
    tot_cap_CUE = 0
    power_mat = []

    # tot_success_prob counts the number of successful samples
    tot_success_prob = 0

    ### IMPORTANT: 2019-02-17
    ### Granuity should be change to include minus 1

    tx_power = tx_power_set
    print(tx_power.shape)

    tx_power = tx_max * np.hstack((tx_power, np.ones((tx_power.shape[0], 1, num_channel))))

    for i in range(num_samples):
        cur_cap = 0
        DUE_mask = 1
        CUE_mask = 1

        for j in range(num_channel):
            cur_ch = channel[i][j]
            cur_ch_cap = cal_SINR_one_sample_one_channel(cur_ch, tx_power[:, :, j], noise)
            cur_cap = cur_cap + cur_ch_cap
            CUE_mask = CUE_mask * (cur_ch_cap[:, num_D2D_user] > CUE_thr)

        for j in range(num_D2D_user):
            DUE_mask = DUE_mask * (cur_cap[:, j] > DUE_thr)
        D2D_sum = np.sum(cur_cap[:, :-1], axis=1)
        CUE_sum = np.sum(cur_cap[:, -1:], axis=1)

        D2D_sum = D2D_sum * CUE_mask * DUE_mask

        max_cap = np.max(D2D_sum)
        max_arg = np.argmax(D2D_sum)
        max_cap_CUE = CUE_sum[max_arg]

        if max_cap != 0:
            tot_success_prob += 1
            power_mat.append(tx_power[max_arg][:-1])
        else:
            power_mat.append(np.zeros(tx_power[0].shape)[:-1])


        tot_cap = tot_cap + max_cap
        tot_cap_CUE = tot_cap_CUE + max_cap_CUE

    return tot_cap / num_samples / num_D2D_user, tot_cap_CUE / num_samples / num_channel, np.array(power_mat)


def optimal_power_check_valid(channel, tx_max, granuty, noise, DUE_thr, CUE_thr):
    num_channel = channel.shape[1]
    ## Note that the num_user i
    num_D2D_user = channel.shape[2] - 1

    ## Feasible
    feasible_channel_mat = []
    infeasible_channel_mat = []

    # tot_success_prob counts the number of successful samples
    tx_power = np.zeros((1, num_D2D_user, num_channel))
    tx_power = tx_max * np.hstack((tx_power, np.ones((tx_power.shape[0], 1, num_channel))))

    for i in range(channel.shape[0]):
        cur_cap = 0
        CUE_mask = 1

        for j in range(num_channel):
            cur_ch = channel[i][j]
            cur_ch_cap = cal_SINR_one_sample_one_channel(cur_ch, tx_power[:, :, j], noise)
            cur_cap = cur_cap + cur_ch_cap
            CUE_mask = CUE_mask * (cur_ch_cap[:, num_D2D_user] > CUE_thr)

        if CUE_mask != 0:
            feasible_channel_mat.append(channel[i])
        else:
            infeasible_channel_mat.append(channel[i])

    return np.array(feasible_channel_mat), np.array(infeasible_channel_mat)


def convert_optimal_power(tx_power_mat, tx_max, Num_power_level, Num_channel):
    num_samples = tx_power_mat.shape[0]
    num_user = tx_power_mat.shape[1]
    resource_alloc = []
    for i in range(num_samples):
        resource_alloc_inner = []
        for j in range(num_user):
            channel_select = np.argmax(tx_power_mat[i, j])
            power_select = np.round(tx_power_mat[i, j, channel_select] / tx_max * (Num_power_level - 1))
            resource_alloc_mat = np.zeros((Num_power_level + Num_channel,))
            resource_alloc_mat[int(power_select)] = 1
            resource_alloc_mat[int(Num_power_level + channel_select)] = 1
            resource_alloc_inner.append(resource_alloc_mat)
        resource_alloc.append(np.array(resource_alloc_inner))
    return np.array(resource_alloc)


def random_power(channel, tx_max, granuty, noise, DUE_thr, CUE_thr, tx_power_set):
    num_sample = channel.shape[0]
    num_channel = channel.shape[1]
    ## Note that the num_user i
    num_D2D_user = channel.shape[2] - 1
    tot_cap = 0
    tot_cap_CUE = 0
    tot_cap_filter = 0
    tot_cap_CUE_vio = 0
    tot_cap_DUE_vio = 0
    tot_cap_CUE_vio_num = 0.001
    tot_cap_DUE_vio_num = 0.001
    DUE_violation = 0
    CUE_violation = 0
    # tot_success_prob counts the number of successful samples
    tot_success_prob = 0

    tx_power = tx_power_set
    tx_power = tx_max * np.hstack((tx_power, np.ones((tx_power.shape[0], 1, num_channel))))

    for i in range(num_sample):
        cur_cap = 0
        DUE_mask = 1
        CUE_mask = 1
        tx_power_select = tx_power[np.random.permutation(tx_power.shape[0])[0]]

        for j in range(num_channel):
            cur_ch = channel[i][j]
            cur_ch_cap = cal_SINR_one_sample_one_channel(cur_ch, tx_power_select[:, j], noise)
            cur_cap = cur_cap + cur_ch_cap
            CUE_mask = CUE_mask * (cur_ch_cap[num_D2D_user] > CUE_thr)
            if cur_ch_cap[num_D2D_user] < CUE_thr:
                tot_cap_CUE_vio = tot_cap_CUE_vio + cur_ch_cap[num_D2D_user]
                tot_cap_CUE_vio_num = tot_cap_CUE_vio_num + 1

        for j in range(num_D2D_user):
            DUE_mask = DUE_mask * (cur_cap[j] > DUE_thr)
            if cur_cap[j] < DUE_thr:
                tot_cap_DUE_vio = tot_cap_DUE_vio + cur_cap[j]
                tot_cap_DUE_vio_num = tot_cap_DUE_vio_num + 1

        D2D_sum = np.sum(cur_cap[:-1])
        CUE_sum = np.sum(cur_cap[-1])
        D2D_sum_filter = D2D_sum * CUE_mask * DUE_mask

        if np.sum(CUE_mask) == 0:
            CUE_violation = CUE_violation + 1

        if np.sum(DUE_mask) == 0:
            DUE_violation = DUE_violation + 1

        max_cap_filter = D2D_sum_filter
        max_cap = D2D_sum
        max_cap_CUE = CUE_sum

        if max_cap_filter != 0:
            tot_success_prob += 1

        tot_cap = tot_cap + max_cap
        tot_cap_CUE = tot_cap_CUE + max_cap_CUE
        tot_cap_filter = tot_cap_filter + max_cap_filter


    CAP_DUE_vio = tot_cap_DUE_vio / tot_cap_DUE_vio_num
    CAP_CUE_vio = tot_cap_CUE_vio / tot_cap_CUE_vio_num
    PRO_DUE_vio = tot_cap_DUE_vio_num / (num_sample * num_D2D_user)
    PRO_CUE_vio = tot_cap_CUE_vio_num / (num_sample * num_channel)
    CAP_TOT = tot_cap / num_D2D_user / num_sample
    CAP_TOT_CUE = tot_cap_CUE / num_channel / num_sample
    CAP_TOT_FIL = tot_cap_filter / num_D2D_user / tot_success_prob

    return CAP_TOT, CAP_TOT_FIL, CAP_TOT_CUE, PRO_CUE_vio, PRO_DUE_vio, CAP_CUE_vio, CAP_DUE_vio


def convert_TX_power(chan_num, user_num, power_granu, tf_output, tx_max):
    ## Output of GCN should be divided proerly.
    ## [power alloc, resource alloc on channel]
    power_mat = np.arange(power_granu) / (power_granu - 1)
    tx_pow_level = tf_output[:, :, :-chan_num] * power_mat
    tx_pow_level_tot = np.sum(tx_pow_level, axis=2) * tx_max
    tx_pow_level_tot = np.reshape(tx_pow_level_tot, [-1, user_num, 1])
    tx_pow_chan = np.multiply(tf_output[:, :, -chan_num:], tx_pow_level_tot)
    return tx_pow_chan


def transform_tx_power(tx_power_mat, Num_power_level, tx_max):
    maximum_val = tx_power_mat.max(axis=2)
    maximum_val = maximum_val.reshape(tx_power_mat.shape[0], tx_power_mat.shape[1], 1)
    selected_power = (tx_power_mat == maximum_val).astype(float)
    return np.round(selected_power * tx_power_mat / tx_max * (Num_power_level - 1)) / (Num_power_level - 1) * tx_max


### 不同的Loss函数

In [ ]:
#####################################################################################################################################################
##训练使用的loss函数
#####################################################################################################################################################
def cal_RATE_tf(channel, tx_power, tx_max, noise, num_samples, log_data_mean, log_data_std):
    chan_num = channel.shape[1]
    user_num = channel.shape[2]
    cap_val = torch.from_numpy(np.array(0.0)).to(torch.float32)
    CUE_cap = []
    
    channel_rev = tf.exp(channel * log_data_std + log_data_mean)

    for i in range(chan_num):
        tx_power_w_CUE = tf.cat([tx_power[:, :, i], tx_max * tf.ones((num_samples, 1))], axis=1)
        tot_ch = tf.multiply(channel_rev[:, i], tx_power_w_CUE.unsqueeze(-1))
        sig_ch = []
        for ch in tot_ch:
            sig_ch.append(tf.diag(ch,0).detach().numpy())
        sig_ch = torch.from_numpy(np.array(sig_ch)).to(torch.float32)

        ###生成一个除了对角线均为0的矩阵 #tf.linalg.diag(sig_ch) ## 由于torch没有这个函数，因此自己构造
        diag_0 = tot_ch
        for i in range(len(diag_0)):
            for j in range(len(diag_0[i])):
                for k in range(len(diag_0[j])):
                    if j != k:
                        diag_0[i][j][k] = 0

        inter_ch = tot_ch - diag_0
        
        inter_ch = tf.sum(inter_ch, axis=1)
        SINR_val = tf.div(sig_ch, inter_ch + noise)
        cap_val = cap_val + tf.log(torch.from_numpy(np.array(1.0)).to(torch.float32) + SINR_val)
        CUE_cap.append(tf.log(torch.from_numpy(np.array(1.0)).to(torch.float32) + SINR_val)[:, -1].detach().numpy())
        
    CUE_cap = torch.from_numpy(np.array(CUE_cap)).to(torch.float32)
    cap_val_D2D = cap_val[:, :user_num - 1]

    return cap_val_D2D, tf.transpose(CUE_cap, 0, 1)


def cal_LOSS_Total_tf(channel, tf_output, noise, DUE_thr, CUE_thr, tx_max, num_samples, log_data_mean, log_data_std,
                      lambda_mat):
    ## Output of GCN should be divided proerly.
    ## [power alloc, resource alloc on channel]
    chan_num = int(channel.shape[1])
    user_num = int(channel.shape[2] - 1)
    power_granu = int(Num_power_level)
    power_mat = torch.from_numpy(np.arange(power_granu) / (power_granu - 1)).to(torch.float32)
    
    tx_pow_level = tf_output[:, :, :power_granu] * power_mat
    tx_pow_level_tot = tf.sum(tx_pow_level, axis=2) * tx_max
    tx_pow_level_tot = tf.reshape(tx_pow_level_tot, [-1, user_num, 1])
    tx_pow_chan = tf.multiply(tf_output[:, :, power_granu:power_granu+chan_num], tx_pow_level_tot)

    D2D_rate, CUE_rate = cal_RATE_tf(channel, tx_pow_chan, tx_max, noise, num_samples, log_data_mean, log_data_std)

    CUE_vio = tf.nn.functional.relu(CUE_thr - CUE_rate) / (CUE_thr + 1e-10)
    integer_vio_1 = -tf.sum(tf.abs(tf_output[:, :, :power_granu+chan_num] - 0.5))
    integer_vio_2 = -tf.sum(tf.abs(tf_output[:, :, power_granu + chan_num:] - 0.5))
    CUE_vio_sum = tf.sum(CUE_vio)

    Loss = - tf.sum(lambda_mat[0] * D2D_rate)  + lambda_mat[1] * CUE_vio_sum + lambda_mat[2] * integer_vio_1 + lambda_mat[3] * integer_vio_2
    return Loss


#####################################################################################################################################################
#####################################################################################################################################################

## 暂留
def cal_EE_tf(cap_val_D2D, tx_power, p_c):
    tx_power_d2d = tf.reduce_sum(tx_power, axis=1)[:, :-1]
    EE = tf.div(cap_val_D2D, tx_power_d2d + p_c)
    return EE

### GCN模型

In [ ]:
import dgl.nn.pytorch as dglnn
import torch.nn as nn
import time

hidden_dim = 400

class Classifier(nn.Module):
    def __init__(self, in_dim, n_classes, n_classes_2):
        super(Classifier, self).__init__()
        self.conv1 = dglnn.GraphConv(in_dim, hidden_dim).to('cuda:0')
        self.conv2 = dglnn.GraphConv(hidden_dim, hidden_dim).to('cuda:0')
        self.norm = nn.BatchNorm1d(hidden_dim).to('cuda:0')
        self.classify = nn.Linear(hidden_dim, n_classes).to('cuda:0')
        self.drop = nn.Dropout(0.02).to('cuda:0')
        self.classify_2 = nn.Linear(hidden_dim, n_classes_2).to('cuda:0')

    def forward(self, g, h):
        # 应用图卷积和激活函数
        h_1_res = self.conv1(g, h) ##Dense
        
        h_1 = F.relu(self.norm(h_1_res))
        h_1 = self.conv2(g, h_1) 
        
        for i in range(6):
            h_1 = self.norm(h_1)
            h_1 += h_1_res ## 残差
            h_1 = F.relu(h_1)
            h_1 = self.drop(h_1)
            h_1 = self.conv2(g, h_1)
        
        h_1_out = self.norm(h_1)
        h_1_out += h_1_res ## 残差
        h_1_out = F.relu(h_1_out)
        ############################
        
        # 应用图卷积和激活函数
        h_2_res = self.conv1(g, h) ##Dense
        h_2 = F.relu(self.norm(h_2_res))
        h_2 = self.conv2(g, h_2) 
        
        for i in range(6):
            h_2 = self.norm(h_2)
            h_2 += h_2_res ## 残差
            h_2 = F.relu(h_2)
            h_2 = self.drop(h_2)
            h_2 = self.conv2(g, h_2)
        
        h_2_out = self.norm(h_2)
        h_2_out += h_2_res ## 残差
        h_2_out = F.relu(h_2_out)
        ############################
        
        
        
        with g.local_scope():
            g.ndata['h_1'] = h_1_out
            # 使用平均读出计算图表示,##一个信息汇聚的过程
            hg = dgl.max_nodes(g, 'h_1')
            final_out = self.classify(hg)

            g.ndata['h_2'] = h_2_out
            # 使用平均读出计算图表示,##一个信息汇聚的过程
            hg_2 = dgl.max_nodes(g, 'h_2')
            final_out_2 = self.classify_2(hg_2)

            
            final_out = torch.reshape(final_out,(len(final_out), 3, 8))
            final_out_2 = torch.reshape(final_out_2,(len(final_out_2), 3, 3))
            
            h_final = torch.cat([final_out,final_out_2],dim=2) ##聚合两个模型的输出3*11
            
            return h_final

### 监督学习训练

In [ ]:
## 正式开始执行
Num_user = 3
Num_channel = 3 
Num_power_level = 8


BW = 1e7
noise = BW*10**-17.4
num_samples_init = int(1e6) 
num_samples_test = int(1e5)
num_init = int(5000)
Size_area = 100   
D2D_dist = 30      
batch_size_set = 1024


DUE_thr = -0.5
tx_max = 10**2.3
channel_error = 0.0

epoch_num_init = 7
epoch_num = 100


CAP_1_MAT_TOT = []
CAP_2_MAT_TOT = []
CAP_CUE_MAT_TOT = []
VIO_PRO_MAT_TOT = []
VIO_CAP_MAT_TOT = []

outer_loop = 0
CUE_thr = 0.6931 * 0.0

#### 1：生成训练集和标签(初始化训练是监督学习)

In [ ]:
## generating all channel
data_train_full = np.array(ch_gen(Size_area, D2D_dist, Num_user, Num_channel, num_samples_init), dtype=np.float32)

## check generated channel 
data_train, data_train_infeasible = optimal_power_check_valid(data_train_full, tx_max, Num_power_level, noise, DUE_thr, CUE_thr)

print("Number of feasible sets:  ", data_train.shape)
print("Number of INfeasible sets:  ", data_train_infeasible.shape)
print("")


## Generate the train channel set
data_train = data_train[:batch_size_set * (data_train.shape[0] // batch_size_set)]
# ## Recalculate the number of feasible solutions
num_samples = data_train.shape[0]
labels = np.zeros(num_samples, )
log_data = np.log(data_train)
log_data_mean = np.mean(log_data)
log_data_std = np.std(log_data)
log_data = (log_data - log_data_mean) / log_data_std

print(log_data.shape)

################################################################################
## Generate train lable
################################################################################
tx_power_set = all_possible_tx_power(Num_channel, Num_user, Num_power_level - 1)
print("Preprecess of data is finished")
time_cur = time.time()
OPT_DUE_train, OPT_CUE_train, opt_power = optimal_power(data_train[:num_init], tx_max, Num_power_level, noise, DUE_thr,
                                                        CUE_thr, tx_power_set)


print("cur_time 1 : ", time.time() - time_cur)
time_cur = time.time()
opt_power_label = convert_optimal_power(opt_power, tx_max, Num_power_level, Num_channel)
print("cur_time 2: ", time.time() - time_cur)

torch.from_numpy(opt_power_label[:,:,:8]).to(torch.int64).argmax(2).size()

#### 2：构建子图(初始化训练是监督学习)

In [ ]:

####step1:三个信道的干涉相拼接
user_num = 3
channel_num = 3

src_index = []
dst_index = []
for i in range(user_num+1):
    for j in range(user_num+1):
        src_index.append(i)
        dst_index.append(j)

print(src_index)
print(dst_index)

####step2:构图，使用Power_Allocation_Dataset_Test_Init是为了和后面测试集、真实训练时构图函数区分开
class Power_Allocation_Dataset_Init(DGLDataset):
    def __init__(self, index_g):
        self.index_g = index_g
        super().__init__(name='Power_Allocation')
    
    def process(self):

        self.graphs, self.labels = [],[]
        
        for index_ in self.index_g:
            
            feature = log_data[index_]
            feat = log_data[index_].reshape((3 * 4 * 4))
            
            all_feat = []
            for _ in range(user_num+1):
                all_feat.append(feat)
            all_feat = np.array(all_feat)
            # edges_data = edges_data_all[index_]
            
            weight_int = []
            ## 三个信道的干涉相加
            for i in range(len(src_index)):
                tmp_weight = 0
                for c in range(channel_num):
                    tmp_weight += feature[c][dst_index[i]][src_index[i]]
                weight_int.append(tmp_weight)
            
            
            node_features = torch.from_numpy(all_feat).to(torch.float32)
            graph_labels = torch.from_numpy(opt_power_label[index_]).to(torch.int64)
            
            ## 将图中边的信息转为Tensor类型
            edge_features = torch.from_numpy(np.array(weight_int))
            edges_src = torch.from_numpy(np.array(src_index)).to(torch.int32)
            edges_dst = torch.from_numpy(np.array(dst_index)).to(torch.int32)

            self.graph = dgl.graph((edges_src, edges_dst), num_nodes=user_num+1) 
            self.graph.ndata['attr'] = node_features
#             self.graph.ndata['label'] = graph_labels ## 无节点label
            self.graph.edata['weight'] = edge_features

    
            ## save sub-figure and graph_label
            self.graphs.append(self.graph.to('cuda:0'))
            self.labels.append(graph_labels.to('cuda:0'))

    def __getitem__(self, i):
        return self.graphs[i], self.labels[i]

    def __len__(self):
        return len(self.graphs)
    

dataset_train = Power_Allocation_Dataset_Init(np.arange(num_init))

In [ ]:
#### step3：batch_size
from dgl.dataloading import GraphDataLoader

batch_size = 128

dataloader_train_init = GraphDataLoader(
    dataset_train,
    batch_size=batch_size, ##决定分为几个batch进行训练
    drop_last=False,
    shuffle=False)

#### 3：训练模型(初始化训练是监督学习)

In [ ]:
## 训练模型
feature_dim = 48

import torch.nn.functional as F
from tqdm import tqdm

model = Classifier(feature_dim, 3*8, 3*3)

opt = torch.optim.Adam(model.parameters(),lr=1e-3) ##weight_decay=5e-4

time_cur = time.time()

val_acc_best = 0
for epoch in tqdm(range(7)):
    all_loss = 0
    for batched_graph, labels in dataloader_train_init: ##fox
#         print(batched_graph)
        feats = batched_graph.ndata['attr']
        logits_ = model(batched_graph, feats)
#         print(logits)
        logits = torch.transpose(logits_,1,2)
        logits_1 = logits[:,:8,:]
        logits_2 = logits[:,8:,:]
        
        logits_1 = F.softmax(logits_1, dim=1)
        logits_2 = F.softmax(logits_2, dim=1)
        
        logits_binary = torch.cat([torch.transpose(logits_1,1,2),torch.transpose(logits_2,1,2)],dim=2)
        
        loss_1 = F.cross_entropy(logits_1, labels[:,:,:8].argmax(2))
        loss_2 = F.cross_entropy(logits_2, labels[:,:,8:].argmax(2))
        loss_3 = -torch.mean(torch.mean(torch.pow(logits_binary-0.5, 4), axis=2))
    
        loss = loss_1+loss_2+loss_3
        all_loss += loss.item()
        opt.zero_grad()
        loss.backward()
        opt.step()
        
    if epoch % 1 == 0:
        print("loss: ", all_loss/len(dataloader_train_init))
        print("Up_to_now_consume_time: ", time.time() - time_cur)        
        
torch.save(model, 'init_network.pth')

### 非监督学习训练

In [ ]:
### 参数设置

print("")
print("Outer_loop: %d " % outer_loop)
print("CUE thr: %0.2f " % (CUE_thr / 0.6931))

lambda_mat = np.ones((4, 1))
coeff_factor = 1.0
lambda_mat[0] = 1.0 * coeff_factor
lambda_mat[1] = 70.0 * coeff_factor
lambda_mat[2] = 15.0 * coeff_factor
lambda_mat[3] = 0.0

lambda_mat = torch.from_numpy(lambda_mat).to(torch.float32).to('cuda:0')

print("lambda")
print(lambda_mat)
print("")

#### 1：构建子图(非监督学习)

In [ ]:
####三个信道的干涉相拼接
user_num = 3
channel_num = 3

src_index = []
dst_index = []
for i in range(user_num+1):
    for j in range(user_num+1):
        src_index.append(i)
        dst_index.append(j)

print(src_index)
print(dst_index)

####构图
class Power_Allocation_Dataset(DGLDataset):
    def __init__(self, index_g, log_data):
        self.index_g = index_g
        self.log_data = log_data
        super().__init__(name='Power_Allocation')
    
    def process(self):

#         self.graphs, self.labels = [],[]
        self.graphs,self.data_org = [],[]
        for index_ in self.index_g:
            
            feature = self.log_data[index_]
            feat = self.log_data[index_].reshape((3 * 4 * 4))
            
            all_feat = []
            for _ in range(user_num+1):
                all_feat.append(feat)
            all_feat = np.array(all_feat)
            # edges_data = edges_data_all[index_]
            
            weight_int = []
            ## 三个信道的干涉相加
            for i in range(len(src_index)):
                tmp_weight = 0
                for c in range(channel_num):
                    tmp_weight += feature[c][dst_index[i]][src_index[i]]
                weight_int.append(tmp_weight)
            
#             print(all_feat)
            node_features = torch.from_numpy(all_feat).to(torch.float32)
#             graph_labels = torch.from_numpy(opt_power_label[index_]).to(torch.int64)
            
            ## 将图中边的信息转为Tensor类型
            edge_features = torch.from_numpy(np.array(weight_int))
            edges_src = torch.from_numpy(np.array(src_index)).to(torch.int32)
            edges_dst = torch.from_numpy(np.array(dst_index)).to(torch.int32)

            self.graph = dgl.graph((edges_src, edges_dst), num_nodes=user_num+1)
            self.graph.ndata['attr'] = node_features
#             self.graph.ndata['label'] = graph_labels ## 无节点label
            self.graph.edata['weight'] = edge_features

            self.feature = torch.from_numpy(feature).to(torch.float32)
            ## save sub-figure and graph_label
            self.graphs.append(self.graph.to('cuda:0'))
            self.data_org.append(self.feature.to('cuda:0'))
#             self.labels.append(graph_labels)

    def __getitem__(self, i):
        return self.graphs[i],self.data_org[i]

    def __len__(self):
        return len(self.graphs)


dataset_train_all = Power_Allocation_Dataset(np.arange(len(log_data)), log_data)
print("*************************完整的训练集构图完毕*************************")

In [ ]:
## batch_size化
batch_size_set = 1024
dataloader_train_true_batch = GraphDataLoader(
    dataset_train_all,
    batch_size=batch_size_set, ##构成一个完整的sample后续使用
    drop_last=False, ## 该参数未知
    shuffle=False)

#### 2：训练模型(非监督学习)

In [ ]:
##训练使用的loss函数
#####################################################################################################################################################
def cal_RATE_tf(channel, tx_power, tx_max, noise, num_samples, log_data_mean, log_data_std):
    chan_num = channel.shape[1]
    user_num = channel.shape[2]
    cap_val = torch.from_numpy(np.array(0.0)).to(torch.float32).to('cuda:0')
    CUE_cap = []
    
    channel_rev = tf.exp(channel * log_data_std + log_data_mean).to('cuda:0')

    for i in range(chan_num):
        tx_power_w_CUE = tf.cat([tx_power[:, :, i], tx_max * tf.ones((num_samples, 1)).to('cuda:0')], axis=1)
        tot_ch = tf.multiply(channel_rev[:, i], tx_power_w_CUE.unsqueeze(-1))
        sig_ch = []
        for ch in tot_ch:
            sig_ch.append(tf.diag(ch,0).cpu().detach().numpy())
        sig_ch = torch.from_numpy(np.array(sig_ch)).to(torch.float32).to('cuda:0')

        ###生成一个除了对角线均为0的矩阵 #tf.linalg.diag(sig_ch) ## 由于torch没有这个函数，因此自己构造
        diag_0 = tot_ch
        for i in range(len(diag_0)):
            for j in range(len(diag_0[i])):
                for k in range(len(diag_0[j])):
                    if j != k:
                        diag_0[i][j][k] = 0
#         print(diag_0)
        inter_ch = tot_ch - diag_0
        
        inter_ch = tf.sum(inter_ch, dim=1)
        SINR_val = tf.div(sig_ch, inter_ch + noise)
        cap_val = cap_val + tf.log(torch.from_numpy(np.array(1.0)).to(torch.float32).to('cuda:0') + SINR_val)
        CUE_cap.append(tf.log(torch.from_numpy(np.array(1.0)).to(torch.float32).to('cuda:0') + SINR_val)[:, -1].cpu().detach().numpy())
        
    CUE_cap = torch.from_numpy(np.array(CUE_cap)).to(torch.float32).to('cuda:0')
    cap_val_D2D = cap_val[:, :user_num - 1]

    return cap_val_D2D, tf.transpose(CUE_cap, 0, 1)


def cal_LOSS_Total_tf(channel, tf_output, noise, DUE_thr, CUE_thr, tx_max, num_samples, log_data_mean, log_data_std,
                      lambda_mat):
    ## Output of DNN should be divided proerly.
    ## [power alloc, resource alloc on channel]
    chan_num = int(channel.shape[1])
    user_num = int(channel.shape[2] - 1)
    power_granu = int(Num_power_level)
    power_mat = torch.from_numpy(np.arange(power_granu) / (power_granu - 1)).to(torch.float32)
#     print('***********')
#     print("power_mat: ", power_mat)
#     print('***********')
    tx_pow_level = tf_output[:, :, :power_granu] * power_mat.to('cuda:0')
#     print("tx_pow_level: ", tx_pow_level)
    tx_pow_level_tot = tf.sum(tx_pow_level, dim=2) * tx_max
    tx_pow_level_tot = tf.reshape(tx_pow_level_tot, [-1, user_num, 1])
    tx_pow_chan = tf.multiply(tf_output[:, :, power_granu:power_granu+chan_num], tx_pow_level_tot)

    D2D_rate, CUE_rate = cal_RATE_tf(channel, tx_pow_chan, tx_max, noise, num_samples, log_data_mean, log_data_std)

    CUE_vio = tf.nn.functional.relu(CUE_thr - CUE_rate) / (CUE_thr + 1e-10)
    integer_vio_1 = -tf.sum(tf.abs(tf_output[:, :, :power_granu+chan_num] - 0.5))
    integer_vio_2 = -tf.sum(tf.abs(tf_output[:, :, power_granu + chan_num:] - 0.5))
    CUE_vio_sum = tf.sum(CUE_vio)

    Loss = - tf.sum(lambda_mat[0] * D2D_rate)  + lambda_mat[1] * CUE_vio_sum + lambda_mat[2] * integer_vio_1 + lambda_mat[3] * integer_vio_2
    return Loss


In [ ]:
cap_full = []
prob_full = []
learning_rate_cur = 1e-6

print("")
print("learning_rate : ", learning_rate_cur)
print("lambda : ", lambda_mat)

######################################################################################################
model = torch.load('init_network.pth')


print("model loading finished")
print("")

opt_2 = torch.optim.Adam(model.parameters(),lr=learning_rate_cur) ##, weight_decay=5e-4

### 开始训练
for i in range(10):
    if not os.path.exists('./' + str(i) + '-saved_model/'): #判断所在目录下是否有该文件名的文件夹
        os.mkdir('./' + str(i) + '-saved_model/') #创建多级目录用mkdirs，单击目录mkdir
        
    for epoch in range(20):
        all_loss = 0
        for batched_graph, data_org in tqdm(dataloader_train_true_batch): ##fox
#             print(batched_graph)
            feats = batched_graph.ndata['attr']
            logits_ = model(batched_graph, feats)
            ## 分别做softmax, 并聚合成3*11
            logits_1 = F.softmax(logits_[:,:,:8], dim=2)
            logits_2 = F.softmax(logits_[:,:,8:], dim=2)
            logits = torch.cat([logits_1,logits_2],dim=2)
            
#             print("data_org.shape: ", data_org.shape)
            
            ## 这里的log_data一定是batch_size大小的，不然无法与ligits匹配
            loss_2 = cal_LOSS_Total_tf(data_org, logits, noise, DUE_thr, CUE_thr, tx_max, batch_size_set, log_data_mean, 
                                     log_data_std, lambda_mat)

            all_loss += loss_2.item()
                
            opt_2.zero_grad()
            loss_2.backward()
            opt_2.step()

        print(str(epoch) + "---------------------------------------------> True_loss: ", all_loss/len(dataloader_train_true_batch))    
        
        if epoch % 1 == 0:
            torch.save(model, './' + str(i) + '-saved_model/' + str(epoch) + '-network.pth')  
    
       

    
    print("")
    print(str(i) + "-> full model training is finished")
    print("")
    torch.save(model, 'Ture-' + str(i) + '-network.pth') ##保存这一次随机生成的数据集训练的参数结果
    model = torch.load('Ture-' + str(i) + '-network.pth')
    
    print("")
    print(str(i) + "-> generating new data for training")
    print("")
    data_train_full = np.array(ch_gen(Size_area, D2D_dist, Num_user, Num_channel, num_samples_init),
                               dtype=np.float32)
    
    data_train, data_train_infeasible = optimal_power_check_valid(data_train_full, tx_max, Num_power_level,
                                                                  noise,
                                                                  DUE_thr, CUE_thr)

    data_train = data_train[:batch_size_set * (data_train.shape[0] // batch_size_set)]

    ## Recalculate the number of feasible solutions
    num_samples = data_train.shape[0]
#     labels = np.zeros(num_samples, )
    log_data = np.log(data_train)
    log_data = (log_data - log_data_mean) / log_data_std
    
    print("")
    print(str(i) + "-> constructing new graph for training")
    print("")
    
    ## 重新构图
    dataset_train_all = Power_Allocation_Dataset(np.arange(len(log_data)), log_data)
    
    batch_size_set = 1024
    
    dataloader_train_true_batch = GraphDataLoader(
    dataset_train_all,
    batch_size=batch_size_set, ##构成一个完整的sample后续使用
    drop_last=False, ## 该参数未知
    shuffle=False)
    
    
    print("%d-th iteration is finished  " % i)
    print("CUE-Thr   ---   %0.2f" % (CUE_thr / 0.6931))
    print("")

## 计算测试集CDF

#### 生成测试集数据

In [ ]:
## Generate the test channel set
## Only the valid channel is used for test
data_test_full = np.array(ch_gen(Size_area, D2D_dist, Num_user, Num_channel, num_samples_test), dtype=np.float32)
data_test, data_test_infeasible = optimal_power_check_valid(data_test_full, tx_max, Num_power_level, noise, DUE_thr,
                                                            CUE_thr)
log_data_test = np.log(data_test)
log_data_test_mean = np.mean(log_data_test)
log_data_test_std = np.std(log_data_test)
log_data_test_infeasible = np.log(data_test_infeasible)
log_data_test = (log_data_test - log_data_test_mean) / log_data_test_std
log_data_test_infeasible = (log_data_test_infeasible - log_data_test_mean) / log_data_test_std
data_test = data_test + channel_error * np.multiply(data_test,
                                                    np.random.randn(data_test.shape[0], Num_channel, Num_user + 1,
                                                                    Num_user + 1))

In [ ]:
## 针对测试集构图
### 构建子图预训练参数
####三个信道的干涉相拼接
user_num = 3
channel_num = 3

src_index = []
dst_index = []
for i in range(user_num+1):
    for j in range(user_num+1):
        src_index.append(i)
        dst_index.append(j)

print(src_index)
print(dst_index)

####构图
class Power_Allocation_Dataset_Test(DGLDataset):
    def __init__(self, index_g):
        self.index_g = index_g
        super().__init__(name='Power_Allocation')
    
    def process(self):

#         self.graphs, self.labels = [],[]
        self.graphs = []
        for index_ in self.index_g:
            
            feature = log_data_test[index_]
            feat = log_data_test[index_].reshape((3 * 4 * 4))
            
            all_feat = []
            for _ in range(user_num+1):
                all_feat.append(feat)
            all_feat = np.array(all_feat)
            # edges_data = edges_data_all[index_]
            
            weight_int = []
            ## 三个信道的干涉相加
            for i in range(len(src_index)):
                tmp_weight = 0
                for c in range(channel_num):
                    tmp_weight += feature[c][dst_index[i]][src_index[i]]
                weight_int.append(tmp_weight)
            
            
            node_features = torch.from_numpy(all_feat).to(torch.float32)
#             graph_labels = torch.from_numpy(opt_power_label[index_]).to(torch.int64)
            
            ## 将图中边的信息转为Tensor类型
            edge_features = torch.from_numpy(np.array(weight_int))
            edges_src = torch.from_numpy(np.array(src_index)).to(torch.int32)
            edges_dst = torch.from_numpy(np.array(dst_index)).to(torch.int32)

            self.graph = dgl.graph((edges_src, edges_dst), num_nodes=user_num+1)
            self.graph.ndata['attr'] = node_features
#             self.graph.ndata['label'] = graph_labels ## 无节点label
            self.graph.edata['weight'] = edge_features

    
            ## save sub-figure and graph_label
            self.graphs.append(self.graph.to('cuda:0'))
#             self.labels.append(graph_labels)

    def __getitem__(self, i):
        return self.graphs[i]

    def __len__(self):
        return len(self.graphs)

In [ ]:
## 生成测试集所需的图
dataset_test = Power_Allocation_Dataset_Test(np.arange(len(log_data_test)))
print("*************************完整的训练集构图完毕*************************")

## batch_size
dataloader_test = GraphDataLoader(
    dataset_test,
    batch_size=batch_size_set,
    drop_last=False,
    shuffle=False)

graph_all_test = [i for i in dataloader_test]

In [ ]:
## 计算测试集CDF所需的函数
from tqdm import tqdm

def cal_rate_NP_CDF(channel, tx_power_in, tx_max, noise, DUE_thr, CUE_thr):
    num_sample = channel.shape[0]
    num_channel = channel.shape[1]

    ## Note that the num_user i
    num_D2D_user = channel.shape[2] - 1
    tx_power = np.hstack((tx_power_in, tx_max * np.ones((tx_power_in.shape[0], 1, num_channel))))
    power_mat = []
    rate_D2D_CDF = []
    rate_CUE_CDF = []

    for i in range(num_sample):
        cur_cap = 0
        CUE_mask = 1
        for j in range(num_channel):
            cur_ch = channel[i][j]
            cur_power = tx_power[i, :, j]
            cur_power = np.array([cur_power])
            cur_ch_cap = cal_SINR_one_sample_one_channel(cur_ch, cur_power, noise)
            cur_cap = cur_cap + cur_ch_cap[0]
            if cur_ch_cap[0, num_D2D_user] < CUE_thr:
                print("<0")
            CUE_mask = CUE_mask * (cur_ch_cap[0, num_D2D_user] > CUE_thr)


        D2D_sum = np.sum(cur_cap[:-1]) * CUE_mask
        CUE_sum = np.sum(cur_cap[-1])

        rate_D2D_CDF.append(D2D_sum / num_D2D_user)
        rate_CUE_CDF.append(CUE_sum / num_channel)

    return np.array(rate_D2D_CDF), np.array(rate_CUE_CDF)


def cal_SINR_one_sample_one_channel(channel, tx_power, noise):
    ## Note that we transpose the channel to
    diag_ch = np.diag(channel)
    inter_ch = channel-np.diag(diag_ch)
    tot_ch = np.multiply(channel, np.expand_dims(tx_power, -1))
    int_ch = np.multiply(inter_ch, np.expand_dims(tx_power, -1))
    sig_ch = np.sum(tot_ch-int_ch, axis=1)
    int_ch = np.sum(int_ch, axis=1)

    SINR_val = np.divide(sig_ch, int_ch+noise)
    cap_val = np.log(1.0+SINR_val)
    return cap_val


def optimal_power_CDF(channel, tx_max, granuty, noise, DUE_thr, CUE_thr, tx_power_set):
    num_channel = channel.shape[1]
    num_D2D_user = channel.shape[2] - 1
    num_samples = channel.shape[0]
    tx_power = tx_power_set
    tx_power = tx_max * np.hstack((tx_power, np.ones((tx_power.shape[0], 1, num_channel))))
    power_mat = []
    rate_D2D_CDF = []
    rate_CUE_CDF = []
    for i in range(num_samples):
        cur_cap = 0
        CUE_mask = 1
        for j in range(num_channel):
            cur_ch = channel[i][j]
            cur_ch_cap = cal_SINR_one_sample_one_channel(cur_ch, tx_power[:, :, j], noise)
            cur_cap = cur_cap + cur_ch_cap
            CUE_mask = CUE_mask * (cur_ch_cap[:, num_D2D_user] > CUE_thr)
        D2D_sum = np.sum(cur_cap[:, :-1], axis=1) #（10648,）
        CUE_sum = np.sum(cur_cap[:, -1:], axis=1)
        D2D_sum = D2D_sum * CUE_mask ##(在满足CUE阈值下的D2D速率和)

        max_cap = np.max(D2D_sum)
        max_arg = np.argmax(D2D_sum)
        max_cap_CUE = CUE_sum[max_arg]##对应D2D_sum最大时对应的CUE组合

        if np.sum(CUE_mask) != 0: ##满足CUE阈值
            power_mat.append(tx_power[max_arg][:-1]) ##(3,3)
            rate_D2D_CDF.append(max_cap/num_D2D_user) ##这里是求D2D的平均速率，由于有三个D2D，因此除以三次
            rate_CUE_CDF.append(max_cap_CUE/num_channel) ##这里由于CUE重复加了num_channel次结果，因此需要做除法
    return np.array(rate_D2D_CDF), np.array(rate_CUE_CDF), np.array(power_mat)


def random_power_CDF(channel, tx_max, granuty, noise, DUE_thr, CUE_thr):
    num_sample = channel.shape[0]
    num_channel = channel.shape[1]
    ## Note that the num_user i
    num_D2D_user = channel.shape[2] - 1
    power_mat = []
    rate_D2D_CDF = []
    rate_CUE_CDF = []

    tx_power = all_possible_tx_power(num_channel, num_D2D_user, granuty)
    tx_power = tx_max * np.hstack((tx_power, np.ones((tx_power.shape[0], 1, num_channel))))

    for i in range(num_sample):
        cur_cap = 0
        CUE_mask = 1
        tx_power_select = tx_power[np.random.permutation(tx_power.shape[0])[0]]

        for j in range(num_channel):
            cur_ch = channel[i][j]
            cur_ch_cap = cal_SINR_one_sample_one_channel(cur_ch, tx_power_select[:, j], noise)
            cur_cap = cur_cap + cur_ch_cap
            # print("cur_ch_cap: ", cur_ch_cap)
            # print("CUE_thr: ", CUE_thr)
            CUE_mask = CUE_mask * (cur_ch_cap[num_D2D_user] > CUE_thr)

        D2D_sum = np.sum(cur_cap[:-1])* CUE_mask
        CUE_sum = np.sum(cur_cap[-1])
        rate_D2D_CDF.append(D2D_sum / num_D2D_user)
        rate_CUE_CDF.append(CUE_sum / num_channel)
        power_mat.append(tx_power_select[:-1])

    return np.array(rate_D2D_CDF), np.array(rate_CUE_CDF), np.array(power_mat)



def Print_GCN_OUT_CDF(model, graph_all_test, log_data, real_data, Num_channel, Num_user, Num_power_level, tx_max, noise, DUE_thr, CUE_thr):
    CDF_granu = 50.0
    
    logits_ = model(graph_all_test[0], graph_all_test[0].ndata['attr'])
    logits_1 = F.softmax(logits_[:,:,:8], dim=2)
    logits_2 = F.softmax(logits_[:,:,8:], dim=2)
    GCN_out_pre = torch.cat([logits_1,logits_2],dim=2).cpu().detach().numpy()
    
    num = 0
    for batch_graph in tqdm(graph_all_test):
        if num >= 1:
            logit_tmp = model(batch_graph, batch_graph.ndata['attr'])
            logits_1 = F.softmax(logit_tmp[:,:,:8], dim=2)
            logits_2 = F.softmax(logit_tmp[:,:,8:], dim=2)
            logit_tmp_cat = torch.cat([logits_1,logits_2],dim=2).cpu().detach().numpy()
            GCN_out_pre = np.concatenate([GCN_out_pre,logit_tmp_cat],axis=0)

        num+=1
    
    print("GCN_out_pre.shape: ", GCN_out_pre.shape)
    
    GCN_out_pre = GCN_out_pre[:, :, :Num_channel + Num_power_level]
    GCN_out = convert_TX_power(Num_channel, Num_user, Num_power_level, GCN_out_pre, tx_max)
    GCN_Filter = transform_tx_power(GCN_out, Num_power_level, tx_max)

    rate_D2D_CDF, rate_CUE_CDF = cal_rate_NP_CDF(real_data, GCN_Filter, tx_max, noise, DUE_thr, CUE_thr)
    sorted_D2D = np.sort(rate_D2D_CDF)
    sorted_CUE = np.sort(rate_CUE_CDF)
    opt_granu = int(sorted_D2D.shape[0] / CDF_granu)
    print("D2D")
    CG_GCN_D2D = sorted_D2D[::opt_granu]
    CG_GCN_CUE = sorted_CUE[::opt_granu]
    print(sorted_D2D[::opt_granu])
    print("")
    print("CUE")
    print(sorted_CUE[::opt_granu])
    print("")
    return GCN_Filter,CG_GCN_D2D,CG_GCN_CUE



def Print_CDF(model_full, graph_all_test, log_data_test, data_test, Num_channel, Num_user, Num_power_level,
                    tx_max, noise, DUE_thr, CUE_thr, tx_power_set):

    CDF_granu = 50.0
    print("***Optimal CDF***")
    rate_D2D_CDF, rate_CUE_CDF, power_mat = optimal_power_CDF(data_test, tx_max, Num_power_level, noise, DUE_thr, CUE_thr, tx_power_set)
    sorted_D2D = np.sort(rate_D2D_CDF)
    sorted_CUE = np.sort(rate_CUE_CDF)
    opt_granu = int(sorted_D2D.shape[0]/CDF_granu)
    
    print("D2D")
    opt_D2D = sorted_D2D[::opt_granu]
    opt_CUE = sorted_CUE[::opt_granu]
    print(sorted_D2D[::opt_granu])
    print("")
    print("CUE")
    print(sorted_CUE[::opt_granu])
    print("")
    print("")

    print("***Full GCN***")
    GCN_tx_power_full,CG_GCN_D2D,CG_GCN_CUE = Print_GCN_OUT_CDF(model_full, graph_all_test, log_data_test, data_test, Num_channel, Num_user, Num_power_level, tx_max, noise, DUE_thr, CUE_thr)
    print("")



    rate_D2D_CDF, rate_CUE_CDF, power_mat = random_power_CDF(data_test, tx_max, Num_power_level, noise, DUE_thr, CUE_thr)
    sorted_D2D = np.sort(rate_D2D_CDF)
    sorted_CUE = np.sort(rate_CUE_CDF)
    opt_granu = int(sorted_D2D.shape[0]/CDF_granu)
    print("***Random CDF***")

    print(opt_granu)
    print(sorted_D2D.shape[0])
    print("D2D")
    Random_D2D = sorted_D2D[::opt_granu]
    Random_CUE = sorted_CUE[::opt_granu]
    print(sorted_D2D[::opt_granu])
    print("")
    print("CUE")
    print(sorted_CUE[::opt_granu])
    print("")
    print("")

    return opt_D2D, opt_CUE, CG_GCN_D2D, CG_GCN_CUE, Random_D2D, Random_CUE

In [ ]:
model.eval() ##声明模型在测试

## 打印CDF
opt_D2D, opt_CUE,CG_GCN_D2D,CG_GCN_CUE,Random_D2D, Random_CUE = Print_CDF(model, graph_all_test,
                                                                          log_data_test, data_test, 
                                                                          Num_channel, Num_user, 
                                                                          Num_power_level, tx_max, noise, 
                                                                          DUE_thr, CUE_thr, tx_power_set)